In [1]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import cv2 #画像処理のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy import signal  #信号処理のためのモジュール
from scipy.stats import skew, kurtosis  #歪度と尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.preprocessing import LabelEncoder  #カテゴリ変数を数値化するためのモジュール
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score, f1_score, fbeta_score#機械学習モデルの性能評価のためのモジュール
import tensorflow as tf #TensorFlow(Googleが開発したオープンソースの機械学習フレームワーク)
from tensorflow import keras  #TensorFlow用のニューラルネットワークライブラリAPI
from tensorflow.keras import layers #ニューラルネットワークのレイヤーを定義するためのモジュール
from keras.utils import np_utils  #one-hot-encodingのためのやつ

In [2]:
#メモ
#画像のサイズは縦：横=1088:1920=17:30

#よく使うgit command
#google colabと揃える
#git fetch origin main
#git reset --hard origin/main

In [2]:
#自分のGoogle Driveとドッキング
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#GPUを使うための処理
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
#マクロの定義
N = 1 #Encoderの層の数
NUM_HEADS = 8 #アテンションヘッドの数
KEY_DIM = 30  #キー次元
DROPOUT = 0.1 #ドロップアウト率
LENGTH = 240  #リサイズ後の画像の横の長さ
HEIGHT = 136  #リサイズ後の画像の縦の長さ

In [5]:
#正解データへのパスとファイル名取得
CollectPath = "/content/drive/MyDrive/DICOMO/正解データ/"
CollectFilename = os.listdir(CollectPath)

In [6]:
#不正解データへのパスとファイル名取得
IncollectPath = "/content/drive/MyDrive/DICOMO/不正解データ/"
IncollectFilename = os.listdir(IncollectPath)

In [10]:
#教師データを格納するリストを定義
img = []

In [11]:
#正解データをグレースケールに変換・リサイズ
for i in CollectFilename:
     image = cv2.imread(CollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(LENGTH, HEIGHT))
     img.append(image)

In [12]:
#不正解データをグレースケールに変換・リサイズ
for i in IncollectFilename:
     image = cv2.imread(IncollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(LENGTH, HEIGHT))
     img.append(image)

In [13]:
img

[array([[154, 249, 250, ..., 133, 136, 127],
        [113, 210, 250, ..., 129, 132, 136],
        [131, 251, 250, ..., 158, 127, 146],
        ...,
        [210, 211, 211, ..., 162, 160, 162],
        [209, 210, 208, ..., 158, 162, 165],
        [207, 207, 207, ..., 145, 166, 159]], dtype=uint8),
 array([[247, 242, 245, ..., 145, 143, 145],
        [242, 241, 243, ..., 146, 143, 150],
        [169, 241, 241, ..., 149, 147, 148],
        ...,
        [200, 201, 202, ..., 157, 157, 161],
        [200, 201, 202, ..., 159, 166, 159],
        [200, 201, 201, ..., 144, 158, 156]], dtype=uint8),
 array([[154, 249, 250, ..., 133, 136, 127],
        [113, 210, 250, ..., 129, 132, 136],
        [131, 251, 250, ..., 158, 127, 146],
        ...,
        [210, 211, 211, ..., 162, 160, 162],
        [209, 210, 208, ..., 158, 162, 165],
        [207, 207, 207, ..., 145, 166, 159]], dtype=uint8),
 array([[154, 249, 250, ..., 133, 136, 127],
        [113, 210, 250, ..., 129, 132, 136],
        [131, 25

In [14]:
#ndarray型に変換
img = np.array(img)

In [15]:
#教師ラベルを作成
#渡れる時を1、渡れない時を0とする
CollectLabel = np.ones(len(CollectFilename))
IncollectLabel = np.zeros(len(IncollectFilename))
label = np.concatenate((CollectLabel, IncollectLabel))

#one-hot-encoding([1, 0]や[0, 1]のようにする)
label = np_utils.to_categorical(label)

In [17]:
# Define the input shape
input_shape = (HEIGHT, LENGTH)
output_shape = (1,)

#形を定義(このモジュールは行列でないとダメっぽい)
x_encoder = layers.Input(shape=input_shape)

In [18]:
#Transformer Encoder Layer(BERT)
for i in range(N):
      #Multi-Head-Attention Layer
      attention_encoder = layers.MultiHeadAttention(num_heads=NUM_HEADS, key_dim=KEY_DIM, use_bias=True)(x_encoder, x_encoder, x_encoder)

      #Dropout Layer
      attention_encoder = layers.Dropout(rate=DROPOUT)(attention_encoder)
      #Add & Norm Layer
      attention_encoder = layers.LayerNormalization()(x_encoder + attention_encoder)

      #Feed-Forward-Network
      ffn_encoder = layers.Dense(LENGTH * 4, use_bias=True, activation="relu")(attention_encoder)
      ffn_encoder = layers.Dense(LENGTH, use_bias=True)(ffn_encoder)

      #Dropout Layer
      ffn_encoder = layers.Dropout(rate=DROPOUT)(ffn_encoder)
      #Add & Norm Layer
      x_encoder = layers.LayerNormalization()(attention_encoder + ffn_encoder)

In [19]:
x = layers.Flatten()(x_encoder)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

In [20]:
model = keras.Model(inputs=x_encoder, outputs=outputs)

In [21]:
# compile model
model.compile(
    loss="categorical_crossentropy",   # 誤差関数
    optimizer="adam",     # 最適化手法
    metrics=['accuracy']
    )

In [33]:
#トレーニングデータとテストデータの分割
x_train, x_test, y_train, y_test = train_test_split(img, label, train_size = 0.8, shuffle = True)

In [34]:
model.fit(x_train, y_train, batch_size=32, epochs=150, shuffle=True, validation_split=0.2)

Epoch 1/150
13/13 [==============================] - 0s 9ms/step - loss: 72.3713 - accuracy: 0.9561 - val_loss: 46.9194 - val_accuracy: 0.9381
Epoch 2/150
13/13 [==============================] - 0s 7ms/step - loss: 71.4732 - accuracy: 0.9406 - val_loss: 34.7031 - val_accuracy: 0.9691
Epoch 3/150
13/13 [==============================] - 0s 6ms/step - loss: 46.5048 - accuracy: 0.9457 - val_loss: 69.8113 - val_accuracy: 0.9175
Epoch 4/150
13/13 [==============================] - 0s 6ms/step - loss: 34.6024 - accuracy: 0.9612 - val_loss: 27.7705 - val_accuracy: 0.9794
Epoch 5/150
13/13 [==============================] - 0s 7ms/step - loss: 41.5426 - accuracy: 0.9509 - val_loss: 153.7024 - val_accuracy: 0.8763
Epoch 6/150
13/13 [==============================] - 0s 6ms/step - loss: 41.2682 - accuracy: 0.9509 - val_loss: 17.1708 - val_accuracy: 0.9897
Epoch 7/150
13/13 [==============================] - 0s 7ms/step - loss: 14.6101 - accuracy: 0.9922 - val_loss: 38.5387 - val_accuracy: 0.958

In [35]:
y_pred = model.predict(x_test)

4/4 [==============================] - 0s 3ms/step


In [36]:
#0.5以上の時に1, 以下の時に0とする
threshold = 0.5
y_pred = np.where(y_pred >= threshold, 1, 0)

In [37]:
accuracy_score(y_test, y_pred)

0.9508196721311475

In [38]:
f1_score(y_test, y_pred, average='micro')

0.9508196721311475